### Consider:
* Rewrite cell 14
* Feature engineering (see results of tree fitting)
* SHAP
* Create a branch with polars. Compare compute performance

In [ ]:
from kagglehub import dataset_load, KaggleDatasetAdapter

import numpy as np
import pandas as pd
import seaborn as sns
import statistics as st
import matplotlib.pyplot as plt

from sklearn.preprocessing import TargetEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import HistGradientBoostingClassifier

import shap

In [ ]:
transactions = dataset_load(
    KaggleDatasetAdapter.PANDAS,
    'ealtman2019/credit-card-transactions',
    'credit_card_transactions-ibm_v2.csv',
)

In [ ]:
def format_col_names(df: pd.DataFrame) -> pd.DataFrame:
    # Format column names: lowercase all characters and remove non-alphabetics
    names = {}
    for col in df.columns:
        names[col] = ''.join(char.lower() for char in col if char.isalpha())
    return df.rename(columns = names)


def clean_data(df: pd.DataFrame) -> pd.DataFrame:

    # Parse hour and minute data from time column
    df['hour'] = df['time'].str[0:2].astype('int64')
    df['minute'] = df['time'].str[3:5].astype('int64')

    # Convert amount to float
    df['amount'] = df['amount'].str.replace('$', '').astype('float')

    # Log transform amount to produce a less skewed distribution
    df['logamount'] = np.log(df['amount'])
    df['logamount'] = df['logamount'].fillna(0)
    df.loc[df['logamount'] == -np.inf, ['logamount']] = 0

    # Convert isfraud to binary
    df['isfraud'] = df['isfraud'].apply(lambda x: 1 if x == 'Yes' else 0)

    return df


def one_hot_encode_categoricals(df: pd.DataFrame) -> pd.DataFrame:
    # One-hot encode usechip, errors
    dummies_usechip = df['usechip'].str.get_dummies()
    dummies_errors = df['errors'].str.get_dummies(sep = ',')
    drop_cols = ['usechip', 'errors']
    return pd.concat([df.drop(columns = drop_cols), dummies_usechip, dummies_errors], axis = 'columns')


def balance_data(df: pd.DataFrame) -> pd.DataFrame:
    # Sample the data, producing a dataset balanced on isfraud
    num_fraud = df.loc[df['isfraud'] == 1].shape[0]
    fraudulent = df.loc[df['isfraud'] == 1].sample(num_fraud)
    not_fraudulent = df.loc[df['isfraud'] == 0].sample(num_fraud)
    return pd.concat([fraudulent, not_fraudulent])


def process_balance(df: pd.DataFrame) -> pd.DataFrame:
    df = format_col_names(df)
    df = clean_data(df)
    df = df.drop(columns = ['user', 'card', 'time', 'amount'])
    df = one_hot_encode_categoricals(df)
    df = format_col_names(df)
    df = balance_data(df)
    return df.sample(frac = 1) # Shuffle data

In [ ]:
def target_encoding(df: pd.DataFrame, target: str) -> tuple[pd.DataFrame, pd.Series]:
    # Target encoding for merchantcity, merchantstate, merchantname, zip, mcc
    # To avoid data leakage, perform this function last (after balancing)
    drop_cols = ['merchantname', 'merchantcity', 'merchantstate', 'zip', 'mcc']
    for col in drop_cols:
        target_encoded = TargetEncoder().fit_transform(df[col].to_numpy().reshape(-1, 1), df[target].to_numpy())
        df = pd.concat([
            df.drop(columns = col),
            pd.DataFrame(target_encoded, columns = ['target_encoded_' + f'{col}'], index = df.index)
            ], axis = 'columns'
        )
    return (df.drop(columns = target), df[target])


def cv_fit_model(model, df: pd.DataFrame, target: str, n_splits: int) -> tuple[list, list]:
    # Fit model with cross-validation
    models, scores = [], []
    kf = KFold(n_splits = n_splits)
    # Perform target encoding independently on each split to avoid data leakage
    for train_ind, test_ind in kf.split(df):
        X_train, y_train = target_encoding(df.iloc[train_ind], target)
        X_test, y_test = target_encoding(df.iloc[test_ind], target)
        model.fit(X_train, y_train)
        models.append(model)
        score = model.score(X_test, y_test)
        scores.append(score)
    return (models, scores)


def median_model(model_scores: tuple[list, list]):
    # Return model with median performance
    median_score = st.median(model_scores[1])
    median_ind = model_scores[1].index(median_score)
    return model_scores[0][median_ind]


def gen_confusion_matrix(model_scores: tuple[list, list], df: pd.DataFrame, target: str):
    # Generate a confusion matrix
    model = median_model(model_scores)
    X, y = target_encoding(df, target)
    return ConfusionMatrixDisplay.from_estimator(model, X, y, normalize = 'true', cmap = 'Blues')


def gen_roc_curve(model_scores: tuple[list, list], df: pd.DataFrame, target: str):
    # Generate a ROC curve
    model = median_model(model_scores)
    X, y = target_encoding(df, target)
    return RocCurveDisplay.from_estimator(model, X, y)

In [ ]:
transactions_processed_balanced = process_balance(transactions)

In [ ]:
# Logistic regression model
logistic_model_scores = cv_fit_model(
    LogisticRegression(solver = 'newton-cholesky'),
    transactions_processed_balanced,
    target = 'isfraud',
    n_splits = 5
)

In [ ]:
gen_confusion_matrix(logistic_model_scores, transactions_processed_balanced, target = 'isfraud')

In [ ]:
gen_roc_curve(logistic_model_scores, transactions_processed_balanced, target = 'isfraud')

In [ ]:
# Decision tree model
tree_model_scores = cv_fit_model(
    DecisionTreeClassifier(ccp_alpha = 0.005),
    transactions_processed_balanced,
    target = 'isfraud',
    n_splits = 5
)

In [ ]:
# Most significant splits:
# x[19]: target_encoded_merchantstate
# x[16]: technicalglitch
# x[20]: target_encoded_zip
# x[17]: target_encoded_merchantname
plot_tree(median_model(tree_model_scores))

In [ ]:
gen_confusion_matrix(tree_model_scores, transactions_processed_balanced, target = 'isfraud')

In [ ]:
gen_roc_curve(tree_model_scores, transactions_processed_balanced, target = 'isfraud')

In [ ]:
# Means of determining appropriate alpha_ccp values:
# https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html#sphx-glr-auto-examples-tree-plot-cost-complexity-pruning-py
'''
target_encoded = target_encoding(transactions_processed_balanced, 'isfraud')
path = median_model(tree_model_scores).cost_complexity_pruning_path(*target_encoded)

ccp_alphas, impurities = path.ccp_alphas, path.impurities
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")
'''

In [ ]:
# Histogram gradient boosting tree model
hgb_model_scores = cv_fit_model(
    HistGradientBoostingClassifier(validation_fraction = None),
    transactions_processed_balanced,
    target = 'isfraud',
    n_splits = 5
)

In [ ]:
gen_confusion_matrix(hgb_model_scores, transactions_processed_balanced, target = 'isfraud')

In [ ]:
gen_roc_curve(hgb_model_scores, transactions_processed_balanced, target = 'isfraud')

In [ ]:
hgb_model = median_model(hgb_model_scores)
explainer = shap.TreeExplainer(hgb_model)